# Notebook for Airconics examples 

IMPORTANT!: ipython notebook bug requires first code cells to be run manually (i.e. not with 'Run All' Cells). After this, all cells can be rerun with 'Run all below'. 

In [1]:
import airconics
%gui qt4
# Initialise the display
from OCC.Display.SimpleGui import init_display
display, start_display, add_menu, add_function_to_menu = init_display()

Using PyQt4


## Airfoils

In [2]:
# Note these sizing variables aren't used yet
LEPoint = [0., 0., 0.] 
ChordLength = 1
Rotation = 0
Twist = 0

AirfoilSeligName = 'dae11'
# SmoothingPasses = 1   #TODO

# Instantiate class to set up a generic airfoil with these basic parameters
Af1 = airconics.primitives.Airfoil(LEPoint, ChordLength, Rotation, Twist,
                        SeligProfile=AirfoilSeligName)
display.DisplayShape(Af1.Shape, update=True);

TypeError: in method 'Geom2d_Geometry_Translated', argument 2 of type 'gp_Vec2d const &'

In [13]:
# Add another one (just to test display):
Af2 = airconics.primitives.Airfoil([0.,1.,2.], ChordLength=2,
                        SeligProfile='b707a')
display.DisplayShape(Af2.Curve, update=True, color='BLUE');

TypeError: __init__() got an unexpected keyword argument 'LEPoint'

In [9]:
# Add NACA 4 digit airfoil:
Af3 = airconics.primitives.Airfoil(LEPoint, ChordLength=3, Naca4Profile='2412', EnforceSharpTE=True)
display.DisplayShape(Af3.Curve, update=True, color='GREEN');

AttributeError: Airfoil instance has no attribute 'Curve'

## Lofted Wing

In [ ]:
import airconics.AirCONICStools as act

# root airfoil:
Af_root = 

# tip airfoil:

act.surface_from_airofils(Af)

In [ ]:
# Write Output STEP file:
act.export_STEPFile([Af1,Af2,Af3], 'airfoils.stp')

 <OCC.TopoDS.TopoDS_Shape; proxy of <Swig Object of type 'TopoDS_Shape *' at 0x7f5fecf07bd0> >


<airconics.primitives.Airfoil instance at 0x7f5fecf0c488>

<OCC.Geom2d.Handle_Geom2d_BSplineCurve; proxy of <Swig Object of type 'Handle_Geom2d_BSplineCurve *' at 0x7f5fecf1e630> >